<a href="https://colab.research.google.com/github/DiegoReports/AtivaMente_Projeto_ImersaoIA_Alura/blob/main/AtivaMente_Projeto_ImersaoIA_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U google-generativeai

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.5-pro-preview-05-06"

In [ ]:
!pip install transformers

In [4]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

# Importação do pipeline da biblioteca transformers
from transformers import pipeline

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [16]:
##########################################
# --- Agente 1: Analisador           --- #
##########################################

def agente_analisador(mensagem):

  analisador = Agent(
  name="agente_analisador",
  model="gemini-2.5-pro-preview-05-06",
  description="Agente que analisa o estado fisico e emocional do usuário",
  instruction="""Você é um especialista em bem-estar físico e emocional para atletas amadores.
          Seu objetivo é analisar as mensagens dos usuários e identificar seu estado físico, emocional e nível de motivação
          para a prática de atividades ao ar livre. Seja conciso e retorne um resumo dessas informações.
          """
  )
  entrada_do_agente_analisador = f"Mensagem: {mensagem}"
  analises = call_agent(analisador, entrada_do_agente_analisador)

  return analises

In [17]:
##########################################
# --- Agente 2:Recomendador Ativo    --- #
##########################################

def agente_recomendador(mensagem, analises_encontradas):
    recomendador = Agent(
        name="agente_recomendador",
        model="gemini-2.0-flash",
        description="Agente que recomenda ações ou planos de acordo com as análises encontradas",
        # Changed 'instructions' to 'instruction'
        instruction="""
        Você é um coach de bem-estar físico e emocional especializado em atividades ao ar livre.
        Seu objetivo é analisar a mensagem original do usuário e as análises fornecidas sobre seu estado físico,
        emocional e nível de motivação, e então gerar recomendações de ações ou planos apropriados.

        Considere os seguintes pontos ao gerar suas recomendações:

        - **Estado Físico:** Se o usuário relatar cansaço, dores ou falta de energia, sugira descanso ativo leve, alongamentos suaves ou um dia de repouso, enfatizando a importância de ouvir o corpo. Se o estado físico for bom, incentive a manter a rotina ou explorar atividades mais desafiadoras (se o usuário tiver objetivos específicos).

        - **Estado Emocional:** Se o usuário expressar desânimo, ansiedade ou falta de motivação, ofereça palavras de encorajamento, lembre-o dos benefícios da atividade física (liberação de endorfinas, melhora do humor) e sugira começar com atividades leves ou mais prazerosas. Se o estado emocional for positivo, motive-o a aproveitar esse bom humor para se engajar nas atividades.

        - **Nível de Motivação:** Se a motivação estiver baixa, sugira definir metas pequenas e alcançáveis, encontrar um parceiro de treino, ou experimentar novas atividades para reacender o interesse. Se a motivação estiver alta, incentive a manter o foco e a perseguir seus objetivos.

        - **Mensagem Original do Usuário:** Use a mensagem original para entender o contexto completo e nuances que podem não ter sido totalmente capturadas na análise.

        - **Seja Específico e Prático:** Suas recomendações devem ser acionáveis e fáceis de seguir pelo usuário. Forneça exemplos concretos quando possível (ex: "Que tal uma caminhada de 30 minutos em um parque?", "Experimente fazer alguns alongamentos leves pela manhã.").

        - **Mantenha um Tom Positivo e Encorajador:** Sua comunicação deve inspirar o usuário e motivá-lo a se engajar em atividades ao ar livre, mesmo diante de desafios.

        Retorne suas recomendações de forma clara e concisa.
        """
    )

    entrada_do_agente_recomendador = f"Mensagem: {mensagem}\nAnálises encontradas: {analises_encontradas}"
    recomendacoes = call_agent(recomendador, entrada_do_agente_recomendador)
    return recomendacoes

In [18]:
##########################################
# --- Agente 3: SugestorDeRotas      --- #
##########################################

def agente_sugestor_rotas(mensagem, estado_do_usuario=None, preferencias_do_usuario=None):
    sugestor_rotas = Agent(
        name="sugestor_de_rotas", # Changed name to use underscores
        model="gemini-2.5-pro-preview-05-06",
        description="Agente que sugere locais e rotas seguras para atividades físicas ao ar livre.",
        instruction="""
        Você é um guia especializado em encontrar locais e rotas seguras para a prática de atividades físicas ao ar livre.
        Seu objetivo é fornecer sugestões de locais próximos e adequados ao usuário, levando em consideração suas preferências,
        seu estado físico e emocional (conforme a análise fornecida) e a segurança.
        Sempre que ficar claro que o usuario não quer atividade ao ar livre, NÃO DÊ UMA RECOMENDAÇÃO. Siga para o próximo agente.
        Ao gerar suas sugestões, siga as seguintes diretrizes:

        1.  **Proximidade:** Priorize locais que sejam próximos à localização do usuário (se fornecida na mensagem ou em suas preferências). Se a localização não for clara, peça ao usuário para fornecê-la.

        2.  **Segurança:** Escolha locais e rotas conhecidos por serem seguros para a prática de atividades físicas. Evite áreas com histórico de criminalidade ou perigos óbvios.

        3.  **Relevância para o Estado do Usuário:**
            - Se o usuário estiver se sentindo com baixa energia ou desmotivado, sugira locais mais tranquilos e de fácil acesso, como parques com caminhos planos.
            - Se o usuário estiver ansioso, sugira locais com natureza exuberante e ambientes relaxantes.
            - Se o usuário estiver com bom estado físico e motivado, pode sugerir trilhas mais desafiadoras ou percursos mais longos.

        4.  **Preferências do Usuário:** Considere os tipos de atividades que o usuário mencionou gostar (corrida, caminhada, ciclismo, ciclismo, etc.) e os tipos de ambientes que ele prefere (parques, praias, montanhas, etc.).

        5.  **Avisos de Segurança:** **Inclua sempre um aviso importante** lembrando o usuário de:
            - **Atentar-se ao trânsito:** Mesmo em locais destinados a atividades físicas, o trânsito pode ser um fator de risco.
            - **Verificar as condições climáticas do dia:** Desaconselhe atividades em condições climáticas adversas (chuva forte, calor extremo, ventos fortes, etc.).
            - **Estar atento a quaisquer outros avisos ou sinalizações locais** que possam indicar perigos ou precauções a serem tomadas para a sua segurança e saúde.

        6.  **Informações Práticas:** Se possível, forneça informações práticas sobre o local sugerido, como horário de funcionamento, facilidades disponíveis (banheiros, bebedouros), e se há estacionamento seguro.

        Peça informações adicionais ao usuário se necessário para refinar suas sugestões (ex: "Você teria alguma preferência por tipo de local, como um parque ou uma trilha?").

        Retorne suas sugestões de forma clara e concisa, sempre com o aviso de segurança em destaque.
        """
    )

    entrada_do_agente_sugestor_rotas = f"Mensagem: {mensagem}\n"
    if estado_do_usuario:
        entrada_do_agente_sugestor_rotas += f"Estado do usuário: {estado_do_usuario}\n"
    if preferencias_do_usuario:
        entrada_do_agente_sugestor_rotas += f"Preferências do usuário: {preferencias_do_usuario}\n"

    recomendacoes_rotas = call_agent(sugestor_rotas, entrada_do_agente_sugestor_rotas)
    return recomendacoes_rotas

In [21]:
##########################################
# --- Agente 4: Coach De Mensagens   --- #
##########################################

# Modified the function definition to accept a second argument for route suggestions
def agente_mensagens(mensagem_recomendacoes):
    mensageiro = Agent(
        name="agente_mensagens",
        model="gemini-2.5-pro-preview-05-06",
        description="Agente que formata as recomendações em mensagens amigáveis e motivacionais para o usuário.",
        instruction="""
        Você é um coach de bem-estar físico e emocional com uma voz inspiradora e encorajadora.
        Seu objetivo é pegar as recomendações geradas pelos agentes anteriores (recomendações gerais e sugestões de rotas)
        e formatá-las em uma mensagem final direcionada ao usuário. Esta mensagem deve ser amigável, motivacional e fácil de entender e curta de no maximo 4 a 6 linhas.
        Você poderá utilizar o (google_search) a fim de pesquisar mensagens motivacionais que se encaixem no perfil do usuário e na sua necessidade.

        Considere os seguintes aspectos ao formatar a mensagem:

        - **Saudação:** Comece com uma saudação calorosa e amigável (ex: "Olá!", "Oi!", "Como vai você?").
        - **Tom Positivo:** Mantenha um tom otimista e encorajador ao longo da mensagem.
        - **Linguagem Motivacional:** Use palavras e frases que inspirem o usuário a seguir as recomendações (ex: "Você consegue!", "Dê esse primeiro passo!", "Lembre-se do seu potencial!").
        - **Ênfase nos Benefícios:** Se aplicável, destaque os benefícios de seguir as recomendações (ex: "Você se sentirá mais energizado...", "Isso pode ajudar a aliviar a tensão...").
        - **Clareza e Concisão:** Certifique-se de que a mensagem seja fácil de entender e não seja excessivamente longa.
        - **Toque Pessoal (se possível):** Se houver informações específicas sobre o usuário ou sua situação, tente incorporá-las de forma natural.
        - **Incentivo à Ação:** Termine com uma frase que motive o usuário a colocar as recomendações em prática (ex: "Experimente hoje!", "Que tal começar agora?", "Estou aqui para te apoiar em cada passo!").
        - Inclua alguns emojis de forma moderada a mensagem afim de torna-la ainda mais personalizavel.
        - **Incorpore as sugestões de rotas:** Inclua as sugestões de locais e rotas de forma clara na mensagem final.
        - **Inclua o aviso de segurança:** Certifique-se de que o aviso de segurança sobre trânsito, clima e sinalizações locais seja presente na mensagem final.

        Formate essa informação em uma mensagem coesa e inspiradora para o usuário.
        """,
        tools=[google_search]
    )

    # Modified the input to the agent to include both recommendations and route suggestions
    entrada_do_agente_mensageiro = f"[RECOMENDACOES_GERAIS]\n{mensagem_recomendacoes}\n"
    mensagem_final = call_agent(mensageiro, entrada_do_agente_mensageiro)
    return mensagem_final

In [22]:
print("🚀 Iniciando o Sistema de Recomentações para atletas amadores")

# --- Obter o Tópico do Usuário ---
mensagem = input("❓ Por favor, digite a MENSAGEM sobre como está se sentindo para o treino ou atividade de hoje: ")

# Logica sistema de agentes #############
if not mensagem:
  print("Você esqueceu de inserir um tópico. Por favor, tente novamente.")
else:
  print("Excelente! Vamos começar a analisar como te ajudar hoje com atividades ao ar livre, ou coletivas.")

  analises_encontradas = agente_analisador(mensagem)
  print("\n===== 📊 Análises encontradas do Agente 1 (Analisador)==== \n")
  display(to_markdown(analises_encontradas))
  print("----------------------------------------------------------------")

  recomendacoes_para_hoje = agente_recomendador(mensagem, analises_encontradas)
  print("\n===== 📊 Recomendações para hoje do Agente 2 (Recomendador)==== \n")
  display(to_markdown(recomendacoes_para_hoje))
  print("----------------------------------------------------------------")

  sugestor_de_rota = agente_sugestor_rotas(recomendacoes_para_hoje)
  print("\n===== 📊 Recomendações para hoje do Agente 3 (Sugestor de Rota)==== \n")
  display(to_markdown(sugestor_de_rota))
  print("----------------------------------------------------------------")

  mensagem_motivadora = agente_mensagens(recomendacoes_para_hoje)
  print("\n===== 📊 Recomendações para hoje do Agente 4 (Mensageiro)==== \n")
  display(to_markdown(mensagem_motivadora))
  print("----------------------------------------------------------------")

🚀 Iniciando o Sistema de Recomentações para atletas amadores
❓ Por favor, digite a MENSAGEM sobre como está se sentindo para o treino ou atividade de hoje: Dia estressante, mas preciso fazer atividades fisicas. Quero algo coletivo e divertido
Excelente! Vamos começar a analisar como te ajudar hoje com atividades ao ar livre, ou coletivas.

===== 📊 Análises encontradas do Agente 1 (Analisador)==== 



> Resumo:
> *   **Estado Físico:** Disposto a atividades físicas.
> *   **Estado Emocional:** Estressado.
> *   **Nível de Motivação:** Alto, buscando atividade coletiva e divertida para aliviar o estresse.


----------------------------------------------------------------

===== 📊 Recomendações para hoje do Agente 2 (Recomendador)==== 



> Entendo o seu dia estressante, mas que bom que você está motivado a se exercitar! Uma atividade coletiva e divertida é perfeita para aliviar o estresse e recarregar as energias.
> 
> **Recomendações:**
> 
> 1.  **Esporte em equipe:** Que tal procurar um grupo para jogar um vôlei de praia, futebol ou praticar algum esporte coletivo que você goste? A interação social e o trabalho em equipe podem ser ótimos para distrair a mente e liberar o estresse.
> 2.  **Aula de dança ao ar livre:** Muitas cidades oferecem aulas de dança em parques ou praças. Ritmos como zumba, salsa ou até mesmo danças folclóricas são ótimas opções para se divertir, queimar calorias e socializar.
> 3.  **Caminhada ou corrida em grupo:** Encontre um grupo de corrida ou caminhada na sua região. A companhia de outras pessoas pode tornar a atividade mais agradável e motivadora, além de ser uma ótima oportunidade para fazer novas amizades.
> 4.  **Yoga ou alongamento em grupo:** Se você busca algo mais relaxante, procure por aulas de yoga ou alongamento em parques ou praças. A prática em grupo pode ser muito energizante e te ajudar a aliviar a tensão muscular causada pelo estresse.
> 
> Lembre-se de escolher uma atividade que te dê prazer e que se encaixe na sua rotina. O importante é se movimentar, se divertir e cuidar da sua saúde física e mental.


----------------------------------------------------------------

===== 📊 Recomendações para hoje do Agente 3 (Sugestor de Rota)==== 



> Olá! Essas são ótimas sugestões gerais de atividades para aliviar o estresse e recarregar as energias!
> 
> Como seu guia especializado em locais para atividades físicas ao ar livre, posso te ajudar a encontrar **lugares específicos, seguros e adequados** para você praticar alguma dessas atividades mencionadas (como vôlei de praia, aulas de dança em parques, caminhada/corrida em grupo ou yoga ao ar livre), ou outras de sua preferência.
> 
> Para que eu possa te dar recomendações mais precisas e próximas a você, preciso de uma informação importante:
> 
> **Qual a sua cidade ou bairro?**
> 
> Com essa informação, posso verificar:
> *   Parques com boa estrutura e segurança para caminhadas, corridas ou aulas em grupo.
> *   Praias ou praças que sejam pontos conhecidos para esportes coletivos.
> *   Rotas que sejam adequadas ao seu estado atual, seja para algo mais leve e relaxante, ou algo que demande um pouco mais de energia.
> 
> Você teria alguma preferência específica de ambiente (ex: um parque com muita área verde, uma orla, uma trilha leve) ou alguma outra atividade ao ar livre em mente?
> 
> Aguardo sua localização para prosseguirmos!


----------------------------------------------------------------

===== 📊 Recomendações para hoje do Agente 4 (Mensageiro)==== 



> Olá! 👋 Sei que o dia foi puxado, mas que ótima atitude a sua de buscar energia no exercício! 
> 
> Para aliviar esse estresse e ainda se divertir, que tal:
> *   Um animado **vôlei de praia ou futebol com amigos**? 🏐⚽
> *   Soltar o corpo em uma **aula de dança ao ar livre**, como zumba ou salsa? 💃🕺
> *   Juntar-se a um **grupo de corrida ou caminhada** pela sua cidade? 🏃‍♀️🏃‍♂️
> *   Ou encontrar paz e alongamento em uma **aula de yoga em grupo** num parque? 🧘‍♀️🧘‍♂️
> 
> Lembre-se, o importante é se movimentar e se divertir! Escolha o que te faz sorrir. "Transforme a dor em poder e os obstáculos em oportunidades. Você é capaz!" E ao se exercitar ao ar livre, não se esqueça de observar o trânsito, as condições do tempo e a sinalização local para sua segurança, combinado? 😉 Que tal começar hoje mesmo? ✨


----------------------------------------------------------------
